This first cell makes sure that torchgan is installed. It's primarily here for use with colab, since colab doesn't have torchgan installed by default.

In [1]:
# from torchGAN tutorial

try:
    import torchgan

    print(f"Existing TorchGAN {torchgan.__version__} installation found")
except ImportError:
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "torchgan"])
    import torchgan

    print(f"Installed TorchGAN {torchgan.__version__}")

Existing TorchGAN v0.1.0 installation found


In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn
import torch.utils.data as tdata
from torch.optim import Adam
import torchvision
import torchvision.datasets as dsets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer

This cell takes care of creating the dataset. Any PNG or JPEG image files are acceptable, as long as their width and height are the same. The models also require that the dimensions of the image are a perfect power of 2, but that can be overcome by scaling the images.

Another thing to note is that the file path provided as root cannot be the exact folder that holds the images. In order for the path to be recognized properly, you must specify the folder that contains the folder with the images.

For copyright reasons, the images that I am training from are not included with the GitHub repo. If you are interested in learning more, the citations for all of the images are included in the bibliography of the accompanying report.

In [3]:
size = 64
channels = 3 # change to 1 if using grayscale
data = ImageFolder(
    root="./data/otherGANGogh/images/landscape/",
    transform=transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        #transforms.Grayscale(),
        transforms.Normalize(mean=.5, std=.5)
    ])
)
print(len(data))

15000


In [4]:
dataloader = tdata.DataLoader(data, batch_size=64, shuffle=True)

In [5]:
# adapted from torchgan tutorial

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_size": size,           # this is how many pixels wide and tall the generated images will be
            "out_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh(),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_size": size,            # this is how many pixels wide and tall we resized the images to be.
            "in_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

In [6]:
# from torchgan tutorial

minimax_losses = [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()]
wgangp_losses = [
    WassersteinGeneratorLoss(),
    WassersteinDiscriminatorLoss(),
    WassersteinGradientPenalty(),
]
lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]

If you're running this on colab, be sure to select a GPU runtime for the best performance.

In [7]:
# more from torchgan tutorial

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 10
else:
    device = torch.device("cpu")
    epochs = 5

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cpu
Epochs: 5


In [24]:
# from torchgan tutorial

trainer = Trainer(
    dcgan_network,
    lsgan_losses,
    sample_size=64,
    epochs=400,             # you can override the number of epochs here if you want to try to get better results
    device=device
)

This is the cell that actually does the training. After it finishes, use the next cell if you want to keep training the same model.

In [11]:
trainer(dataloader)

c:\users\leafy\pycharmprojects\gangogh\venv\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Saving Model at './model/gan0.model'
Epoch 1 Summary
Epoch time duration : 1093.5766220092773
generator Mean Gradients : 15.610295308710686
discriminator Mean Gradients : 82.22784864744416
Mean Running Discriminator Loss : 0.14996620954034176
Mean Running Generator Loss : 0.6748779137717916
Generating and Saving Images to ./images/epoch1_generator.png


c:\users\leafy\pycharmprojects\gangogh\venv\lib\site-packages\torchvision\utils.py:68: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(



Saving Model at './model/gan1.model'
Epoch 2 Summary
Epoch time duration : 302.0853340625763
generator Mean Gradients : 9.316171535907346
discriminator Mean Gradients : 69.06701856266034
Mean Running Discriminator Loss : 0.09922616566392653
Mean Running Generator Loss : 0.6092537719201534
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 3 Summary
Epoch time duration : 294.52990198135376
generator Mean Gradients : 6.831519953119669
discriminator Mean Gradients : 59.359494262194325
Mean Running Discriminator Loss : 0.07946761221697567
Mean Running Generator Loss : 0.5835720143630995
Generating and Saving Images to ./images/epoch3_generator.png

Saving Model at './model/gan3.model'
Epoch 4 Summary
Epoch time duration : 295.37965536117554
generator Mean Gradients : 5.4884254345724255
discriminator Mean Gradients : 50.85141301663774
Mean Running Discriminator Loss : 0.06624936669064249
Mean Running Generator Loss : 0.5722404447166209

Use this cell instead of the previous one if you want to resume training a model that you saved from a previous session. Be sure to use the correct file path and file name when you call the function.

In [25]:
trainer.load_model("./model/gan4.model")
trainer(dataloader)

Loading Model From './model/gan4.model' to cpu
Saving Model at './model/gan0.model'
Epoch 301 Summary
Epoch time duration : 722.0589020252228
generator Mean Gradients : 6.46511217412435
discriminator Mean Gradients : 8.494622940750826
Mean Running Discriminator Loss : 0.033217040129727954
Mean Running Generator Loss : 0.49969406692629664
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 302 Summary
Epoch time duration : 901.6521351337433
generator Mean Gradients : 6.791450095361007
discriminator Mean Gradients : 9.103027983632733
Mean Running Discriminator Loss : 0.03317885702537976
Mean Running Generator Loss : 0.49969091195845683
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 303 Summary
Epoch time duration : 374.27845454216003
generator Mean Gradients : 7.056884646983044
discriminator Mean Gradients : 9.619448774844207
Mean Running Discriminator Loss : 0.033146430859471